In [1]:
import argparse
import time
import json
import csv
import random
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By
with open('facebook_credentials.txt') as file:
    EMAIL = file.readline().split('"')[1]
    PASSWORD = file.readline().split('"')[1]

In [3]:
def _extract_name_reaction(item):
    actualname = item.find_all(class_="darkTouch _1aj5 l")
    names = []  # fix 1: initialize names as a list
    for post in actualname:  # fix 2: change posts to post
        name = post.get('href')
        names.append(name)  # fix 3: append name to the list names
    return names

In [4]:
def _extract_post_id(item):
    postIds = item.find_all(class_="ib cc _1aj4")
    post_id = ""
    for postId in postIds:
        post_id = postId.find(class_='darkTouch _1aj5 l').get('href')
    return post_id


In [6]:
def _extract_reaction(item):
    actualreacts = item.find_all(class_="scrollAreaColumn")
    reaction = ""
    if actualreacts:
        for posts in actualreacts:
            paragraphs =posts.find_all(attrs={'aria-label': True})
            reaction = ""
            for index in range(0, len(paragraphs)):
                reaction += paragraphs[index].get('aria-label')
    return reaction

In [10]:
def _extract_html(bs_data):

    #Add to check
    with open('./bs.html',"w", encoding="utf-8") as file:
        file.write(str(bs_data.prettify()))

    k = bs_data.find_all("div", { "id" : "rootcontainer" })
    postBigDict = list()
    for item in k:
        postDict = dict()
        #postDict['Post'] = _extract_post_text(item)
        #postDict['Comments'] = _extract_comments(item)

        postDict['Name_reaction'] = _extract_name_reaction(item)
         #postDict['PostId'] = _extract_post_id(item)
        postDict['Reaction'] = _extract_reaction(item)
        # postDict['Shares'] = _extract_shares(item)
       

        #Add to check
        postBigDict.append(postDict)
        with open('./postBigDicts.json','w', encoding='utf-8') as file:
            file.write(json.dumps(postBigDict, ensure_ascii=False).encode('utf-8').decode())

        return postBigDict

In [11]:
def _login(browser, email, password):
    browser.get("http://facebook.com")
    browser.maximize_window()
    browser.find_element("name", "email").send_keys(email)
    browser.find_element("name", "pass").send_keys(password)
    browser.find_element("name", "login").click()
    time.sleep(5)


In [12]:
def _count_needed_scrolls(browser, infinite_scroll, numOfPost):
    if infinite_scroll:
        lenOfPage = browser.execute_script(
            "window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;"
        )
    else:
        # roughly 8 post per scroll kindaOf
        lenOfPage = int(numOfPost / 8)
    print("Number Of Scrolls Needed " + str(lenOfPage))
    return lenOfPage

In [13]:
def _scroll(browser, infinite_scroll, lenOfPage):
    lastCount = -1
    match = False

    while not match:
        if infinite_scroll:
            lastCount = lenOfPage
        else:
            lastCount += 1

        # wait for the browser to load, this time can be changed slightly ~3 seconds with no difference, but 5 seems
        # to be stable enough
        time.sleep(5)

        if infinite_scroll:
            lenOfPage = browser.execute_script(
                "window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return "
                "lenOfPage;")
        else:
            browser.execute_script(
                "window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return "
                "lenOfPage;")

        if lastCount == lenOfPage:
            match = True

In [14]:
def tra(browser):    
    tran = "//div[@class='_x7p _3nc8']//a[text()='See Translation']"
    trans = browser.find_elements(By.XPATH, tran)
    for transs in trans:
        action = webdriver.common.action_chains.ActionChains(browser)
        try:
            # move to where the un collapse on is
            action.move_to_element_with_offset(transs, 5, 5)
            action.perform()
            transs.click()
        #    time.sleep(20)
            time.sleep(5)
        except:
        # do nothing right here
            pass 
def getBack(browser):
    if not browser.current_url.endswith('actualPosts'):
        print('redirected!!!')
        browser.back()
        print('got back!!!')
        
def replied (browser):        
#first uncollapse collapsed comments
        unCollapseCommentsButtonsXPath = "//a[text()=' replied']"
        unCollapseCommentsButtons = browser.find_elements(By.XPATH, unCollapseCommentsButtonsXPath)
        for unCollapseComment in unCollapseCommentsButtons:
            action = webdriver.common.action_chains.ActionChains(browser)
            try:
                # move to where the un collapse on is
                action.move_to_element_with_offset(unCollapseComment, 5, 5)
                action.perform()
                unCollapseComment.click()
                time.sleep(5)
            except:
                # do nothing right here
                pass
def com2(browser):             
 #second set comment ranking to show all comments
        rankDropdowns = browser.find_elements_by_class_name('_2pln') #select boxes who have rank dropdowns
        rankXPath = '//div[contains(concat(" ", @class, " "), "uiContextualLayerPositioner") and not(contains(concat(" ", @class, " "), "hidden_elem"))]//div/ul/li/a[@class="_54nc"]/span/span/div[@data-ordering="RANKED_UNFILTERED"]'
        for rankDropdown in rankDropdowns:
            #click to open the filter modal
            action = webdriver.common.action_chains.ActionChains(browser)
            try:
                action.move_to_element_with_offset(rankDropdown, 5, 5)
                action.perform()
                rankDropdown.click()
            except:
                pass

            # if modal is opened filter comments
            ranked_unfiltered = browser.find_elements_by_xpath(rankXPath) # RANKED_UNFILTERED => (All Comments)
            if len(ranked_unfiltered) > 0:
                try:
                    ranked_unfiltered[0].click()
                except:
                    pass   
def com3 (browser):        
#first uncollapse collapsed comments
        unCollapseCommentsButtonsXPath = "//a[@class='touchable primary']"
        unCollapseCommentsButtons = browser.find_elements(By.XPATH, unCollapseCommentsButtonsXPath)
        for unCollapseComment in unCollapseCommentsButtons:
            action = webdriver.common.action_chains.ActionChains(browser)
            try:
                # move to where the un collapse on is
                action.move_to_element_with_offset(unCollapseComment, 5, 5)
                action.perform()
                unCollapseComment.click()
                time.sleep(5)
            except:
                # do nothing right here
                pass    


def react(browser):        
#first uncollapse collapsed comments
        reactt = "//a[@class='_45m8']"
        reactts = browser.find_elements(By.XPATH, reactt)
        for unCollapseComment in reactts:
            action = webdriver.common.action_chains.ActionChains(browser)
            try:
                # move to where the un collapse on is
                action.move_to_element_with_offset(unCollapseComment, 5, 5)
                action.perform()
                unCollapseComment.click()
                time.sleep(10)
            except:
                # do nothing right here
                pass
            
def rand_proxy():
    proxy = random.choice(ips)
    return(proxy)

In [18]:
def extract(page, numOfPost, infinite_scroll=False, scrape_comment=False):
    global postBigDict

    option = Options()
    option.add_argument("--disable-infobars")
    option.add_argument("start-maximized")
    option.add_argument("--disable-extensions")
   # option.add_argument("--headless")

    # Pass the argument 1 to allow and 2 to block
    option.add_experimental_option("prefs", {
        "profile.default_content_setting_values.notifications": 1
    })
    
    
 #  
  # proxy= rand_proxy()
  #  option.add_argument(f'--proxy-server={proxy}')
#
    # chromedriver should be in the same folder as file
    browser = webdriver.Chrome(executable_path="./chromedriver", options=option)
    _login(browser, EMAIL, PASSWORD)
    browser.get(page)
    lenOfPage = _count_needed_scrolls(browser, infinite_scroll, numOfPost)
      
    num_scrolls_to_skip = 368

# Scroll down the page by the height of the viewport times the number of times to skip
    scroll_height = browser.execute_script("return window.innerHeight;")
    for i in range(num_scrolls_to_skip):
        browser.execute_script("window.scrollBy(0, {});".format(scroll_height))
        time.sleep(5)
        
    posts = list()
    for i in range(1):
    
        time.sleep(2)
    
    # Extract the new set of post data from the container element
        new_posts = browser.find_elements(By.XPATH,"//div[@class='_34qc _3hxn _3myz _4b45']//a")
    
    # Append the new posts to the list of posts
        posts += new_posts  
    print(len(posts))
        # Open each post one by one
    postBigDicts = list()
    reactions_dict_list = list()
    links_list = list()
    for i in range(460,len(posts)):
            posts = browser.find_elements(By.XPATH,"//div[@class='_34qc _3hxn _3myz _4b45']//a")
            time.sleep(10)
            if posts[i] in posts:
                posts[i].click()
                time.sleep(5)
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")                
                time.sleep(5)
                react(browser)                
                time.sleep(5)
                browser.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
                for i in range(10):
                    com3 (browser)
                    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);") 
                    time.sleep(5)
                source_data = browser.page_source
   # time.sleep(10)
    # Throw your source into BeautifulSoup and start parsing!
                bs_data = bs(source_data, 'html.parser')
  #  time.sleep(10)
                postBigDict1 = _extract_html(bs_data)
                postBigDicts =postBigDicts + postBigDict1
                print('po')
                browser.back()

                try:
                    current_link = browser.current_url
    # add the current link to the list
                    links_list.append(current_link)
                    
                except:
        # If there are no more posts, break out of the loop
                    print("link")
                browser.back()
    
    postBigDict = []

    # iterate through both lists and create a new dictionary for each iteration
    for i in range(len(postBigDicts)):
        postBigDict1 = {'PostId': links_list[i], 'Name_reaction': postBigDicts[i]['Name_reaction'], 'Reaction': postBigDicts[i]['Reaction']}
        postBigDict.append(postBigDict1)
    browser.close()

    return postBigDict


In [19]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Facebook Page Scraper")
    required_parser = parser.add_argument_group("required arguments")
    required_parser.add_argument('-page', '-p', help="The Facebook Public Page you want to scrape", required=True)
    required_parser.add_argument('-len', '-l', help="Number of Posts you want to scrape", type=int, required=True)
    optional_parser = parser.add_argument_group("optional arguments")
    optional_parser.add_argument('-infinite', '-i',
                                 help="Scroll until the end of the page (1 = infinite) (Default is 0)", type=int,
                                 default=0)
    optional_parser.add_argument('-usage', '-u', help="What to do with the data: "
                                                      "Print on Screen (PS), "
                                                      "Write to Text File (WT) (Default is WT)", default="CSV")

    optional_parser.add_argument('-comments', '-c', help="Scrape ALL Comments of Posts (y/n) (Default is n). When "
                                                         "enabled for pages where there are a lot of comments it can "
                                                         "take a while", default="No")
    args = argparse.Namespace(page='https://m.facebook.com/profile.php?id=100064769104263', len=100 , infinite =0 ,usage="CSV", comments = "no")

    infinite = False
    if args.infinite == 1:
        infinite = True

    scrape_comment = False
    if args.comments == 'y':
        scrape_comment = True

    postBigDict = extract(page=args.page, numOfPost=args.len, infinite_scroll=infinite, scrape_comment=scrape_comment)


    #TODO: rewrite parser
    if args.usage == "WT":
        with open('output.txt', 'w') as file:
            for post in postBigDict:
                file.write(json.dumps(post))  # use json load to recover

    elif args.usage == "CSV":
        if os.path.exists("data.csv"):
            with open('data.csv', 'a',newline='', encoding="utf-8") as csvfile:
                writer = csv.writer(csvfile)
                for post in postBigDict:
                    writer.writerow([post['PostId'],post['Name_reaction'], post['Reaction']])
              
        else:
            with open('data.csv', 'w',newline='', encoding="utf-8") as csvfile:
                writer = csv.writer(csvfile)
           
                writer.writerow(['PostId','Name_reaction','Reaction'])
                for post in postBigDict:
                    writer.writerow([ post['PostId'],post['Name_reaction'], post['Reaction']])
              
    else:
        for post in postBigDict:
            print(post)

    print("Finished")


C:\Users\yassi\AppData\Local\Temp\ipykernel_3856\2492897965.py:21: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path="./chromedriver", options=option)


Number Of Scrolls Needed 12
470
po
po
po
po
po
po
po
po
po
po
Finished
